In [1]:
from sklearn.metrics.pairwise import pairwise_distances
from tensorflow.keras.utils import multi_gpu_model
from model import identity_block, conv_block
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import layers
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [18]:
lr=1e-4
epochs=50
decay=1e-2
num_classes=40

data_dir="/home/kjakkala/neuralwave/data/CSI_preprocessed_40.h5"
intruder_dir="/home/kjakkala/neuralwave/data/CSI_preprocessed_Random.h5"

In [19]:
hf = h5py.File(data_dir, 'r')
X_train = np.expand_dims(hf.get('X_train'), axis=-1)
X_test = np.expand_dims(hf.get('X_test'), axis=-1)
y_train = np.eye(num_classes)[hf.get('y_train')]
y_test =np.eye(num_classes)[hf.get('y_test')]
hf.close()

hf = h5py.File(intruder_dir, 'r')
X_data = np.expand_dims(hf.get('X_data'), axis=-1)
hf.close()

print(X_train.shape, X_data.shape)

(1359, 500, 270, 1) (26, 500, 270, 1)


In [6]:
input_layer = layers.Input(shape=(X_train.shape[1:]))

x = layers.Conv2D(64, (7, 7), strides=(2, 2))(input_layer)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

x = conv_block(x, [64, 256], "relu")
x = identity_block(x, [64, 256], "relu")

x = conv_block(x, [128, 512], "relu")
x = identity_block(x, [128, 512], "relu")

x = conv_block(x, [256, 1024], "relu")
x = identity_block(x, [256, 1024], "relu")

x = conv_block(x, [512, 2048], "relu")
x = identity_block(x, [512, 2048], "relu")

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(num_classes, activation=None)(x)
x = layers.Activation('softmax')(x)

model_base = Model(inputs=input_layer, outputs=x)
model_base.summary()
model = multi_gpu_model(model_base, gpus=3)

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr, decay=decay), metrics=['acc'])
history = model.fit(x=X_train, y=y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500, 270, 1)  0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 247, 132, 64) 3200        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 247, 132, 64) 256         conv2d_21[0][0]                  
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 247, 132, 64) 0           batch_normalization_21[0][0]     
__________________________________________________________________________________________________
max_poolin

Train on 1359 samples, validate on 240 samples
Epoch 1/50
 - 19s - loss: 1.6246 - acc: 0.6402 - val_loss: 5.4917 - val_acc: 0.0250
Epoch 2/50
 - 9s - loss: 0.2338 - acc: 0.9382 - val_loss: 7.2471 - val_acc: 0.0250
Epoch 3/50
 - 9s - loss: 0.1039 - acc: 0.9676 - val_loss: 7.0190 - val_acc: 0.0250
Epoch 4/50
 - 9s - loss: 0.0717 - acc: 0.9823 - val_loss: 0.3359 - val_acc: 0.9167
Epoch 5/50
 - 9s - loss: 0.0347 - acc: 0.9948 - val_loss: 0.0997 - val_acc: 0.9667
Epoch 6/50
 - 9s - loss: 0.0281 - acc: 0.9934 - val_loss: 0.0813 - val_acc: 0.9792
Epoch 7/50
 - 9s - loss: 0.0180 - acc: 0.9993 - val_loss: 0.0808 - val_acc: 0.9708
Epoch 8/50
 - 9s - loss: 0.0119 - acc: 0.9993 - val_loss: 0.0828 - val_acc: 0.9750
Epoch 9/50
 - 9s - loss: 0.0104 - acc: 1.0000 - val_loss: 0.0700 - val_acc: 0.9750
Epoch 10/50
 - 9s - loss: 0.0105 - acc: 0.9993 - val_loss: 0.0712 - val_acc: 0.9792
Epoch 11/50
 - 9s - loss: 0.0115 - acc: 0.9971 - val_loss: 0.0573 - val_acc: 0.9875
Epoch 12/50
 - 9s - loss: 0.0085 - ac

In [22]:
np.argmax(model_base.predict(X_data), axis=-1)

array([31, 31, 31,  6, 31, 31, 31, 31, 31, 31, 38, 15, 19,  4,  6, 19, 16,
       15, 15, 12,  6,  6,  6, 19, 19,  6])